In [1]:
# --- LIBRERIAS ---

import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import glob
import os

In [2]:
# --- CONFIGURACIONES MODIFICABLES ---

# 1. RUTA A LOS DATOS
# Busca todos los archivos que terminen en '*_COMPENSADA.xlsx' en la carpeta 
# especificada y los almacena en una lista
data_path = 'data/raw/piezometers/'
piezometer_files = glob.glob(os.path.join(data_path, '*_COMPENSADA.xlsx'))

# Imprime los archivos encontrados
print(f'Archivos con datos piezometricos encontrados: {len(piezometer_files)}')
print(piezometer_files)

# 2. NOMBRES DE COLUMNAS
# Define un diccionario con nombres de columnas originales y a renombrar
columns_dict = {
    'TEMPERATURE' : 'Temperature_C',
    'NE_m' : 'Depth_m',
    'Cota_m' : 'Static_level_masl'
}

# Define una lista con con nombres de columnas no necesarias
columns_list = [
    'ms',
    'LEVEL',
    'P_baro'
]

# 3. CAMPANAS DE TERRENO
# Define un diccionario con el nombre y rango de fechas de cada campana.
# La fecha de termino es excluyente. Se consideran fechas de trabajo efectivo.
field_campaigns = {
    "May 2024": pd.date_range(start='2024-05-21', end='2024-05-23'),
    "Jul 2024": pd.date_range(start='2024-07-25', end='2024-07-28'),
    "Sep 2024": pd.date_range(start='2024-09-03', end='2024-09-07'),
    "Nov 2024": pd.date_range(start='2024-11-05', end='2024-11-12'),
    "Jan 2025": pd.date_range(start='2025-01-21', end='2025-01-23'),
    "Apr 2025": pd.date_range(start='2025-04-28', end='2025-05-02'),
    "Jul 2025": pd.date_range(start='2025-07-08', end='2025-07-16')
}

# 4. MAYOR INTERVALO DE MEDICION DE LOS DATALOGGERS
max_expected_interval='15min'

Archivos con datos piezometricos encontrados: 6
['data/raw/piezometers\\Data_08_11_2024_SDH2PS01_COMPENSADA.xlsx', 'data/raw/piezometers\\Data_15_07_2025_ SDH1PS01_COMPENSADA.xlsx', 'data/raw/piezometers\\Data_15_07_2025_ SDH1PS02_COMPENSADA.xlsx', 'data/raw/piezometers\\Data_15_07_2025_SDH2PP01_COMPENSADA.xlsx', 'data/raw/piezometers\\Data_15_07_2025_SDH2PS02_COMPENSADA.xlsx', 'data/raw/piezometers\\Data_15_07_2025_SDH2PS03_COMPENSADA.xlsx']


In [3]:
# --- DEFINICION DE FUNCIONES ---

# 1. MANEJO DE FECHAS E INDICE

def timestamp_as_index(df):
    """Convierte las columnas 'Date' y 'Time' en un indice Datetime"""

    # Copia el df original
    df_copy = df.copy()

    # Convierte las columnas 'Date' y 'Time' en strings
    date_str = df_copy['Date'].astype(str)
    time_str = df_copy['Time'].astype(str)

    # Crea la columna 'Timestamps' con formato datetime a partir de 'Date' y 'Time'
    df_copy['Timestamps'] = pd.to_datetime(
        date_str + ' ' + time_str,
        format='%Y-%m-%d %H:%M:%S'
    )

    # Establece 'Timestamps' como indice
    df_copy = df_copy.set_index('Timestamps')

    # Elimina las columnas 'Date' y 'Time'
    df_copy = df_copy.drop(columns=['Date', 'Time'])

    print('Indice datetime establecido')
    return df_copy



# 2. FORMATEO DE COLUMNAS

def format_columns(df):
    """Renombra columnas y elimina las innecesarias"""
    df_formatted = df.rename(columns=columns_dict).drop(columns=columns_list, axis=1, errors='ignore')
    print('Nombres de columnas formateados')
    return df_formatted



# 3. IDENTIFICACION DE DATOS DUPLICADOS

def check_duplicates(df):
    """Revisa e informa sobre datos duplicados en el indice"""

    # Genera una serie booleana que almacena los indices duplicados como True
    duplicates = df.index.duplicated(keep=False)

    # Comprueba si la serie duplicates tiene algun valor True
    if not duplicates.any():
        print('No hay datos duplicados')
    # De haberlos, imprime el total y a cuales registros corresponde
    else:
        print(f'Hay {duplicates.sum()} datos duplicados:')
        print(df[duplicates])



# 4. IDENTIFICACION DE SALTOS REGULARES Y ANOMALOS EN LOS DATOS

def check_discontinuities(df, max_expected_interval):
    """Revisa e informa sobre saltos de tiempo en el indice"""
    
    # Genera una serie Timedelta que almcacena el tiempo transcurrido desde el registro anterior
    intervals = df.index.to_series().diff()

    # Genera una nueva serie con los Timedelta que superan un intervalo maximo esperado
    interval_anomalies = intervals[intervals > pd.Timedelta(max_expected_interval)]

    # Imprime los intervalos mas comunes y su frecuencia
    print(f'\nConteo de intervalos:\n{intervals.value_counts().head()}')

    # Comprueba si la serie interval_anomalies tiene algun valor
    if interval_anomalies.empty:
        print(f'\nNo hay intervalos anomalos')
    # De ser asi, imprime los registros con intervalos anomalos
    else:
        print(f'\nIntervalos anómalos:\n{interval_anomalies}')



# 5. IDENTIFICACION DE DATOS ANOMALOS DURANTE CAMPANAS DE TERRENO

def check_outliers(df, campaign_dates, well_name, campaign_name):
    """Identifica, informa y visualiza outliers en un rango de fechas"""

    # Hace una copia del df y lo filtra a las fechas de la campana de terreno
    df_copy = df.copy()
    df_campaign = df_copy.loc[campaign_dates.min() : campaign_dates.max()- pd.Timedelta(seconds=1)].copy()

    # Si no hay datos durante la campana se detiene la funcion
    if df_campaign.empty:
        print("No se encontraron datos para esta campana")
        return

    # Calcula el z-score de los valores de Temperature_C y Depth_m durante la campana
    df_campaign['z_temp'] = stats.zscore(df_campaign['Temperature_C'])
    df_campaign['z_depth'] = stats.zscore(df_campaign['Depth_m'])

    # Crea un df con registros que tengan z-scores > 3
    outlier_condition = (abs(df_campaign['z_temp']) > 3) | (abs(df_campaign['z_depth']) > 3)
    df_outliers = df_campaign[outlier_condition]

    # Comprueba si el df_outliers tiene algun valor anomalo
    if not df_outliers.empty:
        print("\nOutliers detectados:")
        print(df_outliers[['Temperature_C', 'Depth_m', 'z_temp', 'z_depth']])
    else:
        print("\nNo se encontraron valores con z-Score > 3 en esta campana.")

    # Grafica los valores normalizados de Temperature_C y Depth_m
    title = f"{well_name} - {campaign_name}"
    ax = df_campaign[['z_temp', 'z_depth']].plot(
        figsize=(10, 4),
        title=title,
        grid=True
    )
    ax.axhline(3, color='r', linestyle='--', lw=0.8)
    ax.axhline(-3, color='r', linestyle='--', lw=0.8)
    ax.set_ylabel('Z-Score')
    ax.set_xlabel('')
    plt.show()

In [ ]:
# --- BUCLE DE PROCESAMIENTO ---

# Define una carpeta de salida para los datos limpios y la crea si no existe
output_cleaned_path = 'data/processed/piezometers/cleaned/'
os.makedirs(output_cleaned_path, exist_ok=True)

# Bucle externo: itera sobre cada pozo
# file_path refiere a cada valor de la lista piezometer_files
for file_path in piezometer_files:
    
    # Define el nombre de cada pozo usando el nombre de archivo.
    # Mantiene la penultima cadena de texto, separadas por guion bajo
    base_name = os.path.basename(file_path)
    well_name = base_name.split('_')[-2].strip()
    
    # Imprime un titulo que indica el nombre del pozo, de archivo y rango de fechas
    print("\n" + "="*80)
    print(f"POZO: {well_name}")
    print(f"Archivo: {os.path.basename(file_path)}")
    print("="*80)
 

    # Aplica las funciones de formateo de fechas y columnas
    print("\n--- Lectura y formateo de datos ---")
    try:
        df_raw = pd.read_excel(file_path)
        df_processed = timestamp_as_index(df_raw)
        df_processed = format_columns(df_processed)
        print(f"\nRango de fechas: {df_processed.index.min()} - {df_processed.index.max()}")

    # En caso de error, avisa cual archivo no se pudo procesar y continua el bucle  
    except:
        print(f"ERROR: No se pudo procesar el archivo {file_path}")
        continue

    # Genera un nombre de archivo y asigna ubicacion al dataframe formateado
    output_filename = os.path.join(output_cleaned_path, f'{well_name}_cleaned.csv')
    df_processed.to_csv(output_filename)
    print(f"\nDatos limpios para {well_name} guardados en: {output_filename}")


    # Aplica las funciones de diagnostico de datos aplicables a todo el archivo
    print("\n--- Diagnostico de datos ---")
    check_duplicates(df_processed)
    check_discontinuities(df_processed, max_expected_interval)

    # Bucle interno: itera sobre las campanas de terreno. 
    # name y dates refieren a los pares de valores almacenados en el diccionario field_campaigns
    for campaing_name, campaign_dates in field_campaigns.items():

        # Imprime el nombre de la campana
        print(f"\n--- Diagnostico campana {campaing_name}---")

        # Aplica la funcion de identificacion de outliers
        print("Identificacion de outliers")
        check_outliers(df_processed, campaign_dates, well_name, campaing_name)
    
